# Generate video given a model

### Definitions

In [2]:
from torchvision import transforms
import subprocess
import json
import os
import cv2
from PIL import Image
import torch
import torchaudio
import torchaudio.transforms as audio_T
from models.model import AVENet
from torch.autograd import Variable
import numpy as np
class GetSampleFromJson:
    def __init__(self, json_file, local_dir):
        self.json_file = json_file
        self.local_dir = local_dir
        with open(json_file, 'r') as f:
            data_json = json.load(f)
        self.data = data_json['data']
        self.image_base_path = data_json['image_base_path']
        self.audio_base_path = data_json['audio_base_path']
        self._init_transforms()
        self.AmplitudeToDB = audio_T.AmplitudeToDB()
    
    def _init_transforms(self):
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        self.img_transform = transforms.Compose([
            transforms.Resize(224,Image.BICUBIC),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

    def get_sample(self, index):
        sample = self.data[index]
        image = os.path.join(self.image_base_path, sample['image'])
        audio = os.path.join(self.audio_base_path, sample['wav'])
        return image, audio
    
    def check_sample_in_local(self, index):
        img, aud = self.get_sample(index)
        img_local = os.path.join(self.local_dir + "/frame", img.split('/')[-1])
        aud_local = os.path.join(self.local_dir + "/audio", aud.split('/')[-1])
        return os.path.exists(img_local), os.path.exists(aud_local)
    
    def download_sample(self, index):

        img_rem_path, aud_rem_path = self.get_sample(index)
        if all(self.check_sample_in_local(index)):
            print("Sample already downloaded")
        else:
            print(f"Sample NOT in {self.local_dir} \n-> download  ")
            command = ["scp", "-P", "2122", "asantos@pirineus3.csuc.cat:" + img_rem_path, self.local_dir+"/frame"]
            print(command)
            subprocess.run(command)
            command = ["scp", "-P", "2122", "asantos@pirineus3.csuc.cat:" + aud_rem_path, self.local_dir+"/audio"]
            print(command)
            subprocess.run(command)
            
        img_local_path = os.path.join(self.local_dir+"/frame", img_rem_path.split('/')[-1])
        aud_local_path = os.path.join(self.local_dir+"/audio", aud_rem_path.split('/')[-1])

        return img_local_path, aud_local_path
   
    def load_image(self, image_path):
        img = Image.open(image_path).convert('RGB')
        img = self.img_transform(img)
        return img
    
    def show_image(self, img_path):
        img = cv2.imread(img_path)
        cv2.imshow("image", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def load_audio_to_spec(self, audio_path):
        samples, samplerate = torchaudio.load(audio_path)

        if samples.shape[1] < samplerate * 10:
            n = int(samplerate * 10 / samples.shape[1]) + 1
            samples = samples.repeat(1, n)

        samples = samples[...,:samplerate*10]

        spectrogram  =  audio_T.MelSpectrogram(
                sample_rate=samplerate,
                n_fft=512,
                hop_length=239, 
                n_mels=257,
                normalized=True
            )(samples)
        spectrogram =  self.AmplitudeToDB(spectrogram)
        return spectrogram
    

class MatchmapVideoGenerator:
    def __init__(self,model, device, img, spec,args, matchmap = None):
        self.model = model
        self.device = device
        self.spec = Variable(spec.unsqueeze(0)).to(device, non_blocking=True) if spec.dim() == 3 else Variable(spec).to(device, non_blocking=True)
        self.image = Variable(img.unsqueeze(0)).to(device, non_blocking=True) if img.dim() == 3 else Variable(img).to(device, non_blocking=True)
        self.args = args
        self.matchmap = matchmap
        self.video = None

    def compute_matchmap(self):
        self.model.eval()
        with torch.no_grad():
            img_emb,aud_emb = self.model(self.image.float(), self.spec.float(), self.args)
            img_emb = img_emb.squeeze(0)
            aud_emb = aud_emb.squeeze(0)
            self.matchmap = torch.einsum('ct, chw -> thw',aud_emb,img_emb)
        return self.matchmap
    
    def normalize_img(self, value, vmax=None, vmin=None):
        '''
        Normalize heatmap
        '''
        vmin = value.min() if vmin is None else vmin
        vmax = value.max() if vmax is None else vmax
        if not (vmax - vmin) == 0:
            value = (value - vmin) / (vmax - vmin)  # vmin..vmax
        return value
    
    def get_frame_match(self, img_np, matchmap_np, frame_idx):
        assert img_np.ndim == 3, "img_np should be a 3D numpy array"
        assert matchmap_np.ndim == 3, "matchmap_np should be a 3D numpy array"

        matchmap_i = matchmap_np[frame_idx]
        matchmap_i = cv2.resize(matchmap_i, dsize=(224, 224), interpolation=cv2.INTER_LINEAR)
        matchmap_i = self.normalize_img(matchmap_i)
        matchmap_i_photo = (matchmap_i * 255).astype(np.uint8)
        matchmap_i_photo = cv2.applyColorMap(matchmap_i_photo, cv2.COLORMAP_JET)
        matchmap_i_photo = cv2.addWeighted(matchmap_i_photo, 0.5, img_np, 0.5, 0)
        return matchmap_i_photo
    
    def create_video_f(self,img_np, matchmap_np, output_path="matchmap_video.mp4", fps=1):
        n_frames = matchmap_np.shape[0]
        
        # Make sure img_np is in uint8 format
        if img_np.dtype != np.uint8:
            img_np = (img_np * 255).astype(np.uint8)
        
        # Make sure img_np has correct dimensions (224, 224, 3)
        if img_np.shape[:2] != (224, 224):
            img_np = cv2.resize(img_np, (224, 224))
        
        # Use proper codec for compatibility
        # For better compatibility, try 'avc1' or 'H264' instead of 'mp4v'
        fourcc = cv2.VideoWriter_fourcc(*'avc1') 
        
        # Alternative codec options if 'avc1' doesn't work:
        # fourcc = cv2.VideoWriter_fourcc(*'H264')
        # fourcc = cv2.VideoWriter_fourcc(*'XVID')  # More compatible but lower quality
        
        out = cv2.VideoWriter(output_path, fourcc, fps, (224, 224))
        
        if not out.isOpened():
            print("Failed to create VideoWriter. Trying alternative codec...")
            # Try with different codec
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter(output_path.replace('.mp4', '.avi'), fourcc, fps, (224, 224))
        
        for i in range(n_frames):
            frame = self.get_frame_match(img_np, matchmap_np, i)
            
            # Ensure frame is the correct format
            if frame.dtype != np.uint8:
                frame = (frame * 255).astype(np.uint8)
                
            # Ensure frame has the right shape
            if frame.shape[:2] != (224, 224):
                frame = cv2.resize(frame, (224, 224))
                
            # Verify frame is BGR (OpenCV's default format)
            if len(frame.shape) == 3 and frame.shape[2] == 3:
                out.write(frame)
            else:
                print(f"Warning: Frame {i} has incorrect format. Shape: {frame.shape}")
        
        out.release()
        print(f"Video created at: {output_path}")
        
        # Verify the file was created and has a non-zero size
        if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
            print(f"Success! Video file created: {os.path.getsize(output_path)} bytes")
        else:
            print("Error: Video file was not created properly")
            

    def create_video(self,output_path):
        img_np = self.image[0].cpu().numpy()
        img_np = np.transpose(img_np, (1, 2, 0))
        img_np = self.normalize_img(img_np)
        img_np = (img_np * 255).astype(np.uint8)
        img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        if self.matchmap is None:
            self.matchmap = self.compute_matchmap()

        matchmap_np = self.matchmap.cpu().numpy()
        n_frames = matchmap_np.shape[0]
        self.create_video_f(img_np, matchmap_np, output_path, fps=n_frames/10) # 10 sec duration


    def add_audio_to_video(self, video_path, audio_path):
        """
        Add audio to video using ffmpeg and overwrite the output file if it exists.
        """
        temp_output = "temp_output.mp4"
        
        # Ensure the audio file exists
        if not os.path.exists(audio_path):
            raise Exception("Error: Audio file not found.")

        # Use ffmpeg to merge audio and video
        command = [
            "ffmpeg",
            "-y",  # Overwrite output files without asking
            "-i", video_path,  # Input video
            "-stream_loop", "-1",  # Infinite loop for audio
            "-i", audio_path,  # Input audio
            "-map", "0:v",  # Video stream from first input
            "-map", "1:a",  # Audio stream from second input
            "-c:v", "copy",  # Copy video codec (no re-encoding)
            "-c:a", "libmp3lame",  # Encode audio in mp3 format
            "-t", "10",  # 10 sec duration
            temp_output
        ]
        
        try:
            subprocess.run(command, check=True)
            # Delete the original video file
            os.remove(video_path)
            # Rename the temporary output file to the original video file
            os.rename(temp_output, video_path)
        except subprocess.CalledProcessError:
            print("Error adding audio to video.")


    def create_video_with_audio(self, output_path, audio_path):
        self.create_video(output_path)
        self.add_audio_to_video(output_path, audio_path)

def load_model(ckpt_path, args):
    model = AVENet(args)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    checkpoint = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.to(device)
    return model, device
        

### Demo

In [3]:
import sys
from opts import get_arguments

# Simulate command-line arguments
sys.argv = ['script_name', '--order_3_tensor', '--simtype', 'MISA']

args = get_arguments()

#JSON
split = "val"
json_file = f"garbage/{split}.json"
local_dir_saving = "dir_exp_PlacesAudio"

# Load model
# model_name = "2layers_lr1e-5_epoch100"
# model_name = "2layers_lr1e-5_epoch1"
model_name = "lr1e-5_2layersB256_epoch100"

checkpoint_path = f'garbage/{model_name}.pth.tar'
model,device = load_model(checkpoint_path, args)

#SAMPLE
sample_idx = 9

# Get the image and audio
gs = GetSampleFromJson(json_file,local_dir_saving)
img_local_path, aud_local_path = gs.download_sample(sample_idx)

img = gs.load_image(img_local_path)
spec = gs.load_audio_to_spec(aud_local_path)

# Generate the video
mgv = MatchmapVideoGenerator(model,device,img,spec,args)
# mgv.create_video(local_dir_saving+ f"/mm_{model_name}_{split}_{sample_idx}.mp4")
mgv.create_video_with_audio(local_dir_saving+ f"/mm_{model_name}_{split}_{sample_idx}.mp4", aud_local_path)


/tmp/ipykernel_1355/1936299536.py:29: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  transforms.Resize(224,Image.BICUBIC),
/home/albert/miniconda3/envs/ssl-cpu/lib/python3.8/site-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (257) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


Sample already downloaded
Video created at: dir_exp_PlacesAudio/mm_lr1e-5_2layersB256_epoch100_val_9.mp4
Success! Video file created: 124618 bytes


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.3.0 (conda-forge gcc 11.3.0-19)
  configuration: --prefix=/home/albert/miniconda3/envs/ssl-cpu --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static -

# Class for everything
This class will be able to process the txt file of the desired model, and download the model with the desired weights and maybe retrieve the video

In [15]:
def folder_exists_in_cluster(remote_path):
    # Command to check if the folder exists
    check_command = f"ssh -p 2122 asantos@pirineus3.csuc.cat '[ -d \"{remote_path}\" ] && echo Exists || echo NotExists'"
    result = subprocess.run(check_command, shell=True, capture_output=True, text=True)
    
    return result.stdout.strip() == "Exists"

def count_files_in_folder(remote_path):
    # Command to count files in the folder
    count_command = f"ssh -p 2122 asantos@pirineus3.csuc.cat 'ls -1 \"{remote_path}\" | wc -l'"
    result = subprocess.run(count_command, shell=True, capture_output=True, text=True)

    return int(result.stdout.strip()) if result.stdout.strip().isdigit() else 0

def list_files_in_remote_folder(remote_path):
    # Command to list all files in the remote folder
    list_command = f"ssh -p 2122 asantos@pirineus3.csuc.cat 'ls -1 \"{remote_path}\"'"
    result = subprocess.run(list_command, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        return result.stdout.strip().split('\n') if result.stdout.strip() else []
    else:
        print(f"Error listing files in remote folder: {result.stderr}")
        return []

def download_remote_file(remote_path, local_path):
    """
    Downloads a specific file from a remote folder to a local folder using scp.
    """
    # Ensure the local directory exists
    local_dir = os.path.dirname(local_path)
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    # Command to download the specific file
    download_command = f"scp -P 2122 asantos@pirineus3.csuc.cat:\"{remote_path}\" \"{local_path}\""
    result = subprocess.run(download_command, shell=True, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"Successfully downloaded {remote_path} to {local_path}")
    else:
        print(f"Error downloading {remote_path}: {result.stderr}")
# print(folder_exists_in_cluster(remote_path="code"))
# print(count_files_in_folder(remote_path="code"), type(count_files_in_folder(remote_path="code")))
print((list_files_in_remote_folder("models/SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV")[0]))

links_SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV_135932.json


In [5]:
model_name = "try_json"
folder_exists_in_cluster(f'models/{model_name}')

True

In [29]:
from datetime import datetime, timedelta

class ModelOutputRetriever:
    def __init__(self,model_name,local_dir):
        self.model_name = model_name
        self.model_dir = os.path.join("models",model_name)
        self.local_dir = local_dir
        #Check if the folder exists
        if not folder_exists_in_cluster(self.model_dir):
            raise Exception(f"Model {model_name} doesn't exist among models")
        
        # - if more than one ask the user which one if not use that one
        filenames = list_files_in_remote_folder(self.model_dir)
        if len(filenames) > 1:
            request_string = f"There's more than one json for {model_name}\nChoose one:\n"
            for idx, flname in enumerate(filenames):
                file_string = f"[{str(idx)}] {flname}\n"
                request_string = request_string + file_string
            idx_to_process = int(input(request_string))

            print(f"Using [{str(idx_to_process)}] {filenames[idx_to_process]}")

        else:
            idx_to_process = 0
            print(f"Using {filenames[0]}")
        
        #check if json is in local if not download
        json_file = filenames[idx_to_process]
        if not self.check_in_local(json_file):
            download_remote_file(
                remote_path = os.path.join(self.model_dir,json_file),
                local_path = os.path.join(self.local_dir,json_file)
                ) 
        self.json_file = os.path.join(self.local_dir,json_file)
        
        with open(self.json_file, 'r') as f:
            self.data_json = json.load(f)

        self.check_if_available()
        

    def check_in_local(self,file):
        return os.path.exists(os.path.join(self.local_dir,file))
            

    def get_number_of_epochs(self):
        return len(self.data_json) - 1
    
    def get_parameters(self):
        return self.data_json['parameters']
    
    def get_weigths_link(self, epoch):
        return self.data_json[str(epoch)]['weights_link']
    
    def download_weigths(self, epoch):
        weigths_link = self.get_weigths_link(epoch)
        weigths_name = self.model_name + "-" + weigths_link.split('/')[-1]
        if not self.check_in_local(weigths_name):
            print("Downloading weigths")
            download_remote_file(
                remote_path = weigths_link,
                local_path = os.path.join(self.local_dir,weigths_name)
            )
        else:
            print("Weights already in local dir")
        return os.path.join(self.local_dir,weigths_name)
    
    def get_video_link(self, epoch):
        return self.data_json[str(epoch)]['video_link']
    
    def download_video(self, epoch, path_2_save = None):
        video_link = self.get_video_link(epoch)
        sample_idx = self.data_json["parameters"]["val_video_idx"]
        video_name = f'mm_{self.model_name}-epoch{epoch}_val_{sample_idx}.mp4'
        if not self.check_in_local(video_name):
            print("Downloading video")
            download_remote_file(
                remote_path = video_link,
                local_path = os.path.join(path_2_save if path_2_save else self.local_dir
                                          ,video_name)
            )
        else:
            print("Video already in local dir")
        return os.path.join(self.local_dir,video_name)
    
    def check_if_available(self):
        """
        Check if 7 seven days have passed since the time creation
        """
        # Parse the time_creation string into a datetime object
        creation_time = datetime.strptime(self.data_json["parameters"]["time_creation"], "%Y-%m-%d %H:%M")

        # Check if 7 days have passed since the creation time
        if datetime.now() - creation_time > timedelta(days=7):
            print("The model is no longer available.")
            return False
        else:
            print("The model is still available. Creation time: ",creation_time)
            time_difference = timedelta(days=7) - (datetime.now() - creation_time)
            days = time_difference.days
            hours = time_difference.seconds // 3600

            if days > 0:
                print(f"The model will be available for {days} more days.")
            else:
                print(f"The model will be available for {hours} more hours.")
            return True
    
      
model_name = "SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV"
local_dir_saving = "garbage"
mor = ModelOutputRetriever(model_name,local_dir_saving)
print(mor.get_number_of_epochs())
print(mor.get_parameters())
print(mor.get_weigths_link(1))
print(mor.download_video(59,"dir_exp_PlacesAudio"))
print(mor.download_weigths(59))

        

        

Using links_SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV_135932.json
The model is still available. Creation time:  2025-03-25 18:32:00
The model will be available for 6 more days.
59
{'time_creation': '2025-03-25 18:32', 'learning_rate': 1e-05, 'batch_size': 128, 'simtype': 'SISA', 'temperature': 0.07, 'val_video_idx': 10}
/scratch/upftfg03/asantos/135932/SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV/model/epoch1.pth.tar
Successfully downloaded /scratch/upftfg03/asantos/135932/SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV/img/val_videos/epoch_59.mp4 to dir_exp_PlacesAudio/mm_SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV-epoch59_val_10.mp4
garbage/mm_SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV-epoch59_val_10.mp4


KeyboardInterrupt: 

In [23]:
model_name2 = "try_json"
mor2= ModelOutputRetriever(model_name2,local_dir_saving)

Using [2] links_try_json_135914.json
Successfully downloaded models/try_json/links_try_json_135914.json to garbage/links_try_json_135914.json
The model is still available.


# The Mega Function

In [34]:
import sys
from opts import get_arguments


def inference_maker(model_name, epoch, split, sample_idx, local_dir_saving):
    inference_name = f'mm_{model_name}-epoch{epoch}_{split}_{sample_idx}.mp4'
    inference_local_path = os.path.join(local_dir_saving, inference_name)

    if os.path.exists(inference_local_path):
        print("Inference already in local")
        return inference_local_path
    print("Proceed to make the inference")
    model_weights_path = os.path.join(local_dir_saving,
                                      f'{model_name}-epoch{epoch}.pth.tar')
    
    # Simulate command-line arguments for loading the model
    sys.argv = ['script_name', '--order_3_tensor', '--simtype', 'MISA']

    args = get_arguments()

    #Check if the model exist in local
    if os.path.exists(model_weights_path):
        print("The model is in local")
        model, device = load_model(model_weights_path,args)
        print("-Model Loaded")
    else:
        #Request if user wants to download the model
        user_input = input("Model weights not found locally. Do you want to download them? (yes/no): ").strip().lower()
        if user_input == "yes":
            mor = ModelOutputRetriever(model_name, local_dir_saving)
            model_weights_path = mor.download_weigths(epoch)
            model, device = load_model(model_weights_path, args)
            print("-Model Loaded")
        else:
            raise FileNotFoundError("Model weights are required but not available locally.")
    
    #Check if the audio and the image of the sample index is in local
    json_file = f'garbage/{split}.json'
    if os.path.exists(json_file):
        gs = GetSampleFromJson(json_file, local_dir_saving)
        img_local_path, aud_local_path = gs.download_sample(sample_idx)
    else:
        raise FileNotFoundError("json file not found for retrieving the samples")

    img = gs.load_image(img_local_path)
    spec = gs.load_audio_to_spec(aud_local_path)

    mgv = MatchmapVideoGenerator(model, device, img, spec, args)
    mgv.create_video_with_audio(inference_local_path, aud_local_path)
    
    
    return inference_local_path


In [36]:
model_name = "SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV"

inference_maker(
    model_name=model_name,
    epoch=50,
    split="val",
    sample_idx=17,
    local_dir_saving="dir_exp_PlacesAudio"
    )

Proceed to make the inference
The model is in local
-Model Loaded
Sample NOT in dir_exp_PlacesAudio 
-> download  
['scp', '-P', '2122', 'asantos@pirineus3.csuc.cat:/data/upftfg03/asantos/PlacesAudio_400k_distro/images256/p/picnic_area/gsun_0782ffe2a06a466eb3b1d44049b92c89.jpg', 'dir_exp_PlacesAudio/frame']


/tmp/ipykernel_1355/1936299536.py:29: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  transforms.Resize(224,Image.BICUBIC),


['scp', '-P', '2122', 'asantos@pirineus3.csuc.cat:/data/upftfg03/asantos/PlacesAudio_400k_distro/wavs/3/utterance_35672.wav', 'dir_exp_PlacesAudio/audio']


/home/albert/miniconda3/envs/ssl-cpu/lib/python3.8/site-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (257) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


Video created at: dir_exp_PlacesAudio/mm_SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV-epoch50_val_17.mp4
Success! Video file created: 172201 bytes


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.3.0 (conda-forge gcc 11.3.0-19)
  configuration: --prefix=/home/albert/miniconda3/envs/ssl-cpu --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1674566204550/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static -

'dir_exp_PlacesAudio/mm_SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV-epoch50_val_17.mp4'